# Import library

In [5]:
# dataframe
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from mpl_toolkits.mplot3d import Axes3D

# clustering
from sklearn.cluster import AgglomerativeClustering
from sklearn.cluster import KMeans, DBSCAN
import scipy.cluster.hierarchy as shc

# model preparation
from sklearn import preprocessing as ppr
from sklearn.utils import resample
from sklearn.model_selection import train_test_split

# modeling
from sklearn import tree
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import VotingClassifier   
from sklearn.decomposition import PCA
from sklearn.model_selection import GridSearchCV
from sklearn.feature_selection import RFE

# model evaluation
from sklearn.model_selection import cross_val_score
from sklearn.metrics import *

from openai import OpenAI
import openai

In [6]:
import os
from dotenv import load_dotenv
dotenv_path = "../backend/.env"
load_dotenv(dotenv_path=dotenv_path)

True

In [7]:
import os

OPENAI_API_KEY = os.environ.get("OPENAI_API_KEY", default=None)

In [8]:
def generate_conversation_text(conversations):
    conversation_text = ""
    for conversation in conversations:
        role = conversation.get("role", "user")
        content = conversation.get("content", "")
        conversation_text += f"{role}: {content}\n"
    return conversation_text


In [49]:
history = [{"role": "user", "content": "Hi, my name is Alex. I earn 100k a year and need a loan for my property."}, 
           {"role": "assistant", "content": "Could you please provide the loan amount you are requesting?"}, 
           {"role": "user", "content": "Sure, I need 200,000."}, 
                  {"role": "assistant", "content": "Thank you! Could you also share the property value and the purpose of the loan?"}, 
                  {"role": "user", "content": "My property is worth 600,000, and the loan is for home improvement."}, 
                  {"role": "assistant", "content": "Thanks! What is the loan term in months?"}
]
generate_conversation_text(history)

'user: Hi, my name is Alex. I earn 100k a year and need a loan for my property.\nassistant: Could you please provide the loan amount you are requesting?\nuser: Sure, I need 200,000.\nassistant: Thank you! Could you also share the property value and the purpose of the loan?\nuser: My property is worth 600,000, and the loan is for home improvement.\nassistant: Thanks! What is the loan term in months?\n'

In [50]:

examples = [
    {
        "messages": [
                        {"role": "system", "content": "Given the mandatory fields: \"name\", \"income\", \"loan_amount\", \"property_value\", \"loan_term\", and \"loan_purpose\", validate and collect missing or invalid fields. Return a JSON object with the completed fields."}, 
                        {"role": "user", "content": "Hi, I'm John, and I earn 80000 annually. I want a loan of 250000 for my new home."}, 
                        {"role": "assistant", "content": "{\"name\": \"John\", \"income\": 80000, \"loan_amount\": 250000, \"property_value\": null, \"loan_term\": null, \"loan_purpose\": \"Home purchase\"}"}
                    ]
    },

    {"messages": [{"role": "system", "content": "Given the mandatory fields: \"name\", \"income\", \"loan_amount\", \"property_value\", \"loan_term\", and \"loan_purpose\", validate and collect missing or invalid fields. Return a JSON object with the completed fields."}, 
                  {"role": "user", "content": "I’m Sarah, and I need a loan for 150000. My property is valued at 500000. The loan term will be 30 years, and it’s for home improvement."}, 
                  {"role": "assistant", "content": "{\"name\": \"Sarah\", \"income\": null, \"loan_amount\": 150000, \"property_value\": 500000, \"loan_term\": 360, \"loan_purpose\": \"Home improvement\"}"}]},
    
    {"messages": [{"role": "system", "content": "Given the mandatory fields: \"name\", \"income\", \"loan_amount\", \"property_value\", \"loan_term\", and \"loan_purpose\", validate and collect missing or invalid fields. Return a JSON object with the completed fields."}, 
                  {"role": "user", "content": "I’m Sarah, and I need a loan for èé. My property is valued at 500000. The loan term will be 30 years, and it’s for home improvement."}, 
                  {"role": "assistant", "content": "{\"name\": \"Sarah\", \"income\": null, \"loan_amount\": null, \"property_value\": 500000, \"loan_term\": 360, \"loan_purpose\": \"Home improvement\"}"}]},

    {"messages": [{"role": "system", "content": "Given the mandatory fields: \"name\", \"income\", \"loan_amount\", \"property_value\", \"loan_term\", and \"loan_purpose\", validate and collect missing or invalid fields. Return a JSON object with the completed fields."}, 
                  {"role": "user", "content": "My name is Alex, and I earn 120k a year. I need 200k for a cash-out refinancing on my property worth 650000."}, 
                  {"role": "assistant", "content": "{\"name\": \"Alex\", \"income\": 120000, \"loan_amount\": 200000, \"property_value\": 650000, \"loan_term\": null, \"loan_purpose\": \"Cash-out refinancing\"}"}]},

    {"messages": [{"role": "system", "content": "Given the mandatory fields: \"name\", \"income\", \"loan_amount\", \"property_value\", \"loan_term\", and \"loan_purpose\", validate and collect missing or invalid fields. Return a JSON object with the completed fields."}, 
                  {"role": "user", "content": "Hi, I’m Peter. I don’t know my exact income, but it’s around 50000. I want a refinance loan of 300000. My house is worth 700000."}, 
                  {"role": "assistant", "content": "{\"name\": \"Peter\", \"income\": 50000, \"loan_amount\": 300000, \"property_value\": 700000, \"loan_term\": null, \"loan_purpose\": \"Refinance\"}"}]},

    {"messages": [{"role": "system", "content": "Given the mandatory fields: \"name\", \"income\", \"loan_amount\", \"property_value\", \"loan_term\", and \"loan_purpose\", validate and collect missing or invalid fields. Return a JSON object with the completed fields."}, 
                  {"role": "user", "content": "I’m Lisa. I want to buy a new home, and my income is 95000. I’m looking for a loan of 400000 for a property worth 1.2 million over 30 years."}, 
                  {"role": "assistant", "content": "{\"name\": \"Lisa\", \"income\": 95000, \"loan_amount\": 400000, \"property_value\": 1200000, \"loan_term\": 360, \"loan_purpose\": \"Home purchase\"}"}]},

    {"messages": [{"role": "system", "content": "Given the mandatory fields: \"name\", \"income\", \"loan_amount\", \"property_value\", \"loan_term\", and \"loan_purpose\", validate and collect missing or invalid fields. Return a JSON object with the completed fields."}, 
                  {"role": "user", "content": "My name is John. My income is invalid as I don’t have a fixed one."}, 
                  {"role": "assistant", "content": "{\"name\": \"John\", \"income\": null, \"loan_amount\": null, \"property_value\": null, \"loan_term\": null, \"loan_purpose\": null}"}]},

    {"messages": [{"role": "system", "content": "Given the mandatory fields: \"name\", \"income\", \"loan_amount\", \"property_value\", \"loan_term\", and \"loan_purpose\", validate and collect missing or invalid fields. Return a JSON object with the completed fields."}, 
                  {"role": "user", "content": "I’m Emma. I don’t need a loan amount yet, but my house is valued at 900000."}, 
                  {"role": "assistant", "content": "{\"name\": \"Emma\", \"income\": null, \"loan_amount\": null, \"property_value\": 900000, \"loan_term\": null, \"loan_purpose\": null}"}]},

    {"messages": [{"role": "system", "content": "Given the mandatory fields: \"name\", \"income\", \"loan_amount\", \"property_value\", \"loan_term\", and \"loan_purpose\", validate and collect missing or invalid fields. Return a JSON object with the completed fields."}, 
                  {"role": "user", "content": "I’m Bob, and my annual income is 85000. I want to refinance my property worth 400000. I need 100000 over 15 years."}, 
                  {"role": "assistant", "content": "{\"name\": \"Bob\", \"income\": 85000, \"loan_amount\": 100000, \"property_value\": 400000, \"loan_term\": 180, \"loan_purpose\": \"Refinance\"}"}]},
    
    # property express 
    {
      "messages": [
        {"role": "system", "content": "Given the mandatory fields: \"name\", \"income\", \"loan_amount\", \"property_value\", \"loan_term\", and \"loan_purpose\", validate and collect missing or invalid fields. Return a JSON object with the completed fields."},
        {"role": "user", "content": "The last appraisal valued my home at 600,000, and I’d like a 200,000 loan."},
        {"role": "assistant", "content": "{\"name\": null, \"income\": null, \"loan_amount\": 200000, \"property_value\": 600000, \"loan_term\": null, \"loan_purpose\": null}"}
      ]
    },
    {
      "messages": [
        {"role": "system", "content": "Given the mandatory fields: \"name\", \"income\", \"loan_amount\", \"property_value\", \"loan_term\", and \"loan_purpose\", validate and collect missing or invalid fields. Return a JSON object with the completed fields."},
        {"role": "user", "content": "My property is valued at six hundred thousand dollars."},
        {"role": "assistant", "content": "{\"name\": null, \"income\": null, \"loan_amount\": null, \"property_value\": null, \"loan_term\": null, \"loan_purpose\": null}"}
      ]
    },  
    {
      "messages": [
        {"role": "system", "content": "Given the mandatory fields: \"name\", \"income\", \"loan_amount\", \"property_value\", \"loan_term\", and \"loan_purpose\", validate and collect missing or invalid fields. Return a JSON object with the completed fields."},
        {"role": "user", "content": "I’m not sure, but I think my property is worth around 450,000."},
        {"role": "assistant", "content": "{\"name\": null, \"income\": null, \"loan_amount\": null, \"property_value\": 450000, \"loan_term\": null, \"loan_purpose\": null}"}
      ]
    },
    {
      "messages": [
        {"role": "system", "content": "Given the mandatory fields: \"name\", \"income\", \"loan_amount\", \"property_value\", \"loan_term\", and \"loan_purpose\", validate and collect missing or invalid fields. Return a JSON object with the completed fields."},
        {"role": "user", "content": "My property’s value is somewhere between 400,000 and 450,000."},
        {"role": "assistant", "content": "{\"name\": null, \"income\": null, \"loan_amount\": null, \"property_value\": null, \"loan_term\": null, \"loan_purpose\": null}"}
      ]
    },
    {
      "messages": [
        {"role": "system", "content": "Given the mandatory fields: \"name\", \"income\", \"loan_amount\", \"property_value\", \"loan_term\", and \"loan_purpose\", validate and collect missing or invalid fields. Return a JSON object with the completed fields."},
        {"role": "user", "content": "The property is valued at around 600,000 based on the market trend."},
        {"role": "assistant", "content": "{\"name\": null, \"income\": null, \"loan_amount\": null, \"property_value\": 600000, \"loan_term\": null, \"loan_purpose\": null}"}
      ]
    },
    # income
    {
      "messages": [
        {"role": "system", "content": "Given the mandatory fields: \"name\", \"income\", \"loan_amount\", \"property_value\", \"loan_term\", and \"loan_purpose\", validate and collect missing or invalid fields. Return a JSON object with the completed fields."},
        {"role": "user", "content": "I make seventy thousand dollars a year."},
        {"role": "assistant", "content": "{\"name\": null, \"income\": 70000, \"loan_amount\": null, \"property_value\": null, \"loan_term\": null, \"loan_purpose\": null}"}
      ]
    },
    {
      "messages": [
        {"role": "system", "content": "Given the mandatory fields: \"name\", \"income\", \"loan_amount\", \"property_value\", \"loan_term\", and \"loan_purpose\", validate and collect missing or invalid fields. Return a JSON object with the completed fields."},
        {"role": "user", "content": "My income is about 50k."},
        {"role": "assistant", "content": "{\"name\": null, \"income\": 50000, \"loan_amount\": null, \"property_value\": null, \"loan_term\": null, \"loan_purpose\": null}"}
      ]
    },
    {
      "messages": [
        {"role": "system", "content": "Given the mandatory fields: \"name\", \"income\", \"loan_amount\", \"property_value\", \"loan_term\", and \"loan_purpose\", validate and collect missing or invalid fields. Return a JSON object with the completed fields."},
        {"role": "user", "content": "I think my income is roughly 60,000 annually."},
        {"role": "assistant", "content": "{\"name\": null, \"income\": 60000, \"loan_amount\": null, \"property_value\": null, \"loan_term\": null, \"loan_purpose\": null}"}
      ]
    },
    {
      "messages": [
        {"role": "system", "content": "Given the mandatory fields: \"name\", \"income\", \"loan_amount\", \"property_value\", \"loan_term\", and \"loan_purpose\", validate and collect missing or invalid fields. Return a JSON object with the completed fields."},
        {"role": "user", "content": "I’m a teacher, and I make around 45,000 per year."},
        {"role": "assistant", "content": "{\"name\": null, \"income\": 45000, \"loan_amount\": null, \"property_value\": null, \"loan_term\": null, \"loan_purpose\": null}"}
      ]
    },
    {
      "messages": [
        {"role": "system", "content": "Given the mandatory fields: \"name\", \"income\", \"loan_amount\", \"property_value\", \"loan_term\", and \"loan_purpose\", validate and collect missing or invalid fields. Return a JSON object with the completed fields."},
        {"role": "user", "content": "My net income after taxes is 55,000."},
        {"role": "assistant", "content": "Thank you. Could you provide your gross annual income (before taxes)?"}
      ]
    },
    # home purpose
    {
      "messages": [
        {"role": "system", "content": "Given the mandatory fields: \"name\", \"income\", \"loan_amount\", \"property_value\", \"loan_term\", and \"loan_purpose\", validate and collect missing or invalid fields. Return a JSON object with the completed fields."},
        {"role": "user", "content": "I earn $50,000 per year and need a $100,000 loan for repairs."},
        {"role": "assistant", "content": "{\"name\": null, \"income\": 50000, \"loan_amount\": 100000, \"property_value\": null, \"loan_term\": null, \"loan_purpose\": \"Home improvement\"}"}
      ]
    },
    {
    "messages": [
        {"role": "system", "content": "Given the mandatory fields: \"name\", \"income\", \"loan_amount\", \"property_value\", \"loan_term\", and \"loan_purpose\", validate and collect missing or invalid fields. Return a JSON object with the completed fields."},
        {"role": "assistant", "content": "{\"name\": null, \"income\": null, \"loan_amount\": 300000, \"property_value\": 800000, \"loan_term\": null, \"loan_purpose\": null}"}
    ]
    },
    {
      "messages": [
        {"role": "system", "content": "Given the mandatory fields: \"name\", \"income\", \"loan_amount\", \"property_value\", \"loan_term\", and \"loan_purpose\", validate and collect missing or invalid fields. Return a JSON object with the completed fields."},
        {"role": "user", "content": "I’m looking for a loan to purchase a new house."},
        {"role": "assistant", "content": "{\"name\": null, \"income\": null, \"loan_amount\": null, \"property_value\": null, \"loan_term\": null, \"loan_purpose\": \"Home purchase\"}"}
      ]
    },
    {
      "messages": [
        {"role": "system", "content": "Given the mandatory fields: \"name\", \"income\", \"loan_amount\", \"property_value\", \"loan_term\", and \"loan_purpose\", validate and collect missing or invalid fields. Return a JSON object with the completed fields."},
        {"role": "user", "content": "I want to refinance my mortgage."},
        {"role": "assistant", "content": "{\"name\": null, \"income\": null, \"loan_amount\": null, \"property_value\": null, \"loan_term\": null, \"loan_purpose\": \"Refinance\"}"}
      ]
    },
    {
      "messages": [
        {"role": "system", "content": "Given the mandatory fields: \"name\", \"income\", \"loan_amount\", \"property_value\", \"loan_term\", and \"loan_purpose\", validate and collect missing or invalid fields. Return a JSON object with the completed fields."},
        {"role": "user", "content": "I need a loan to cash out some equity from my home."},
        {"role": "assistant", "content": "{\"name\": null, \"income\": null, \"loan_amount\": null, \"property_value\": null, \"loan_term\": null, \"loan_purpose\": \"Cash-out refinancing\"}"}
      ]
    },
    {
      "messages": [
        {"role": "system", "content": "Given the mandatory fields: \"name\", \"income\", \"loan_amount\", \"property_value\", \"loan_term\", and \"loan_purpose\", validate and collect missing or invalid fields. Return a JSON object with the completed fields."},
        {"role": "user", "content": "I need the loan to fix my roof and make some house repairs."},
        {"role": "assistant", "content": "{\"name\": null, \"income\": null, \"loan_amount\": null, \"property_value\": null, \"loan_term\": null, \"loan_purpose\": \"Home improvement\"}"}
      ]
    },
    {
      "messages": [
        {"role": "system", "content": "Given the mandatory fields: \"name\", \"income\", \"loan_amount\", \"property_value\", \"loan_term\", and \"loan_purpose\", validate and collect missing or invalid fields. Return a JSON object with the completed fields."},
        {"role": "user", "content": "I’m not sure yet, but I might need the loan for some house-related expenses."},
        {"role": "assistant", "content": "{\"name\": null, \"income\": null, \"loan_amount\": null, \"property_value\": null, \"loan_term\": null, \"loan_purpose\": null}"}
      ]
    },
    {
      "messages": [
        {"role": "system", "content": "Given the mandatory fields: \"name\", \"income\", \"loan_amount\", \"property_value\", \"loan_term\", and \"loan_purpose\", validate and collect missing or invalid fields. Return a JSON object with the completed fields."},
        {"role": "user", "content": "I’m planning to upgrade my kitchen, so I need some funds for that."},
        {"role": "assistant", "content": "{\"name\": null, \"income\": null, \"loan_amount\": null, \"property_value\": null, \"loan_term\": null, \"loan_purpose\": \"Home improvement\"}"}
      ]
    },
    {
      "messages": [
        {"role": "system", "content": "Given the mandatory fields: \"name\", \"income\", \"loan_amount\", \"property_value\", \"loan_term\", and \"loan_purpose\", validate and collect missing or invalid fields. Return a JSON object with the completed fields."},
        {"role": "user", "content": "I need this loan for personal reasons."},
        {"role": "assistant", "content": "{\"name\": null, \"income\": null, \"loan_amount\": null, \"property_value\": null, \"loan_term\": null, \"loan_purpose\": \"Other purpose\"}"}
      ]
    },
    {
      "messages": [
        {"role": "system", "content": "Given the mandatory fields: \"name\", \"income\", \"loan_amount\", \"property_value\", \"loan_term\", and \"loan_purpose\", validate and collect missing or invalid fields. Return a JSON object with the completed fields."},
        {"role": "user", "content": "I need a loan for both paying off my existing mortgage and some renovations."},
        {"role": "assistant", "content": "{\"name\": null, \"income\": null, \"loan_amount\": null, \"property_value\": null, \"loan_term\": null, \"loan_purpose\": \"Refinance\"}"}
      ]
    },
    {
      "messages": [
        {"role": "system", "content": "Given the mandatory fields: \"name\", \"income\", \"loan_amount\", \"property_value\", \"loan_term\", and \"loan_purpose\", validate and collect missing or invalid fields. Return a JSON object with the completed fields."},
        {"role": "user", "content": "I need a loan, but I haven’t decided on the specifics yet."},
        {"role": "assistant", "content": "{\"name\": null, \"income\": null, \"loan_amount\": null, \"property_value\": null, \"loan_term\": null, \"loan_purpose\": null}"}
      ]
    },
    # loan term
    {
      "messages": [
        {"role": "system", "content": "Given the mandatory fields: \"name\", \"income\", \"loan_amount\", \"property_value\", \"loan_term\", and \"loan_purpose\", validate and collect missing or invalid fields. Return a JSON object with the completed fields."},
        {"role": "user", "content": "I want a term of about 25 years because I’ll retire in 30 years."},
        {"role": "assistant", "content": "{\"name\": null, \"income\": null, \"loan_amount\": null, \"property_value\": null, \"loan_term\": 300, \"loan_purpose\": null}"}
      ]
    },
    {
      "messages": [
        {"role": "system", "content": "Given the mandatory fields: \"name\", \"income\", \"loan_amount\", \"property_value\", \"loan_term\", and \"loan_purpose\", validate and collect missing or invalid fields. Return a JSON object with the completed fields."},
        {"role": "user", "content": "I’d like to have this loan fully paid off by 2053."},
        {"role": "assistant", "content": "{\"name\": null, \"income\": null, \"loan_amount\": null, \"property_value\": null, \"loan_term\": null, \"loan_purpose\": null}"}
      ]
    },
    {
      "messages": [
        {"role": "system", "content": "Given the mandatory fields: \"name\", \"income\", \"loan_amount\", \"property_value\", \"loan_term\", and \"loan_purpose\", validate and collect missing or invalid fields. Return a JSON object with the completed fields."},
        {"role": "user", "content": "I’d like the standard loan term for a mortgage."},
        {"role": "assistant", "content": "{\"name\": null, \"income\": null, \"loan_amount\": null, \"property_value\": null, \"loan_term\": 360, \"loan_purpose\": null}"}
      ]
    },
    {
      "messages": [
        {"role": "system", "content": "Given the mandatory fields: \"name\", \"income\", \"loan_amount\", \"property_value\", \"loan_term\", and \"loan_purpose\", validate and collect missing or invalid fields. Return a JSON object with the completed fields."},
        {"role": "user", "content": "I’m not sure, maybe around 20 years."},
        {"role": "assistant", "content": "{\"name\": null, \"income\": null, \"loan_amount\": null, \"property_value\": null, \"loan_term\": 240, \"loan_purpose\": null}"}
      ]
    },
    {
      "messages": [
        {"role": "system", "content": "Given the mandatory fields: \"name\", \"income\", \"loan_amount\", \"property_value\", \"loan_term\", and \"loan_purpose\", validate and collect missing or invalid fields. Return a JSON object with the completed fields."},
        {"role": "user", "content": "The loan term I’m considering is 360 months."},
        {"role": "assistant", "content": "{\"name\": null, \"income\": null, \"loan_amount\": null, \"property_value\": null, \"loan_term\": 360, \"loan_purpose\": null}"}
      ]
    },
    {
        "messages": [{"role": "system", "content": "Given the mandatory fields: \"name\", \"income\", \"loan_amount\", \"property_value\", \"loan_term\", and \"loan_purpose\", validate and collect missing or invalid fields. "
                   "Return a JSON object with the completed fields."},
                   {"role": "user", "content":  
                        "Chat History : "
                        "user: Hi, my name is Alex. I earn 100k a year and need a loan for my property.\n"
                        "assistant: Could you please provide the loan amount you are requesting?\n"
                        "user: Sure, I need 200,000.\n"
                        "assistant: Thank you! Could you also share the property value and the purpose of the loan?\n"
                        "user: My property is worth 600,000, and the loan is for home improvement.\n"
                        "assistant: Thanks! What is the loan term in months?\n"
                        "Latest user message: I'd like a term of 20 years."
                   }, 
                   {"role": "assistant", "content": "{\"name\": \"Alex\", \"income\": 100000, \"loan_amount\": 200000, \"property_value\": 600000, \"loan_term\": 240, \"loan_purpose\": \"Home improvement\"}"}]
    },
    {
        "messages": [
        {
            "role": "system",
            "content": "Given the mandatory fields: \"name\", \"income\", \"loan_amount\", \"property_value\", \"loan_term\", and \"loan_purpose\", validate and collect missing or invalid fields. "
                    "Return a JSON object with the completed fields."},
        {
            "role": "user", 
            "content":         
                      "Chat History : "
                      "user: I want to refinance my house.\n"
                      "assistant: Sure! Could you please provide your name and income?\n"
                      "user: My name is Susan, and my income is 85,000 annually.\n"
                      "assistant: Thank you. What is the loan amount you are requesting, and what is your property value?\n"
                      "user: The loan amount is 120,000, and my property value is 400,000.\n"
                      "assistant: Got it. Lastly, how long would you like the loan term to be in months?\n"
                      "Latest user message:  I prefer a term of 15 years."
                  },
        {"role": "assistant", "content": "{\"name\": \"Susan\", \"income\": 85000, \"loan_amount\": 120000, \"property_value\": 400000, \"loan_term\": 180, \"loan_purpose\": \"Refinance\"}"}]
    },
    {
        "messages": [
        {
            "role": "system",
            "content": "Given the mandatory fields: \"name\", \"income\", \"loan_amount\", \"property_value\", \"loan_term\", and \"loan_purpose\", validate and collect missing or invalid fields. "
                    "Return a JSON object with the completed fields."},
        {
            "role": "user", 
            "content":         
                    "Chat History : "
                    "user: I need a loan, but I’m not sure of all the details yet.\n"
                    "assistant: No problem! Could you start by providing your name?\n"
                    "user: Sure, it’s Liam.\n"
                    "assistant: Thanks, Liam. Do you know your annual income?\n"
                    "user: Yes, it’s 70,000.\n"
                    "assistant: Great. Do you have an idea of the loan amount you need and the property value?\n"
                    "user: I think I’ll need 150,000, and my property is valued at 500,000.\n"
                    "assistant: Thank you. What is the purpose of the loan?\n"
                    "Latest user message: It’s for refinancing."},
        {"role": "assistant", "content": "{\"name\": \"Liam\", \"income\": 70000, \"loan_amount\": 150000, \"property_value\": 500000, \"loan_term\": 360, \"loan_purpose\": \"Refinance\"}"}]
    },
    {
    "messages": [
      {
        "role": "system",
        "content": "Given the mandatory fields: \"name\", \"income\", \"loan_amount\", \"property_value\", \"loan_term\", and \"loan_purpose\", validate and collect missing or invalid fields. "
                   "Return a JSON object with the completed fields."},
      {
          "role": "user", 
          "content": 
                   "Chat History : "
                   "user: I’m looking to buy a new home and need a loan.\n"
                   "assistant: Got it. Can you start by sharing your name?\n"
                   "user: I’m Kate.\n"
                   "assistant: Thanks, Kate. What is your annual income?\n"
                   "user: I earn 95,000 per year.\n"
                   "assistant: What loan amount are you requesting?\n"
                   "user: I need 300,000 for a property worth 800,000.\n"
                   "assistant: Thank you. What is the loan term in months?\n"
                   "Latest user message:I’d like a term of 25 years."
      },
      {"role": "assistant", "content": "{\"name\": \"Kate\", \"income\": 95000, \"loan_amount\": 300000, \"property_value\": 800000, \"loan_term\": 300, \"loan_purpose\": \"Home purchase\"}"}
    ]
  },
  {
    "messages": [
      {
        "role": "system",
        "content": "Given the mandatory fields: \"name\", \"income\", \"loan_amount\", \"property_value\", \"loan_term\", and \"loan_purpose\", validate and collect missing or invalid fields. "
                   "Return a JSON object with the completed fields."},
      {
          "role": "user", 
          "content": 
                   "Chat History : "
                   "user: Hi, I’m not sure how to calculate my income accurately, but I think I earn around 60,000. Can I still apply for a loan?\n"
                   "assistant: Yes, you can! Let’s proceed. Could you confirm your name?\n"
                   "user: It’s Tom.\n"
                   "assistant: Thank you, Tom. What loan amount are you looking for?\n"
                   "user: About 180,000 for a home improvement project. The property is valued at 450,000.\n"
                   "assistant: Got it. What is the loan term you are considering?\n"
                   "Latest user message:I think 20 years would work."
      },
      {"role": "assistant", "content": "{\"name\": \"Tom\", \"income\": 60000, \"loan_amount\": 180000, \"property_value\": 450000, \"loan_term\": 240, \"loan_purpose\": \"Home improvement\"}"}
    ]
  },
  {
    "messages": [
      {
        "role": "system",
        "content": "Given the mandatory fields: \"name\", \"income\", \"loan_amount\", \"property_value\", \"loan_term\", and \"loan_purpose\", validate and collect missing or invalid fields. "
                   "Return a JSON object with the completed fields."},
      {
          "role": "user", 
          "content": 
                   "Chat History : "
                   "user: Hi, my name is John. I’d like a loan for home improvement.\n"
                   "assistant: Hi John! Could you provide your annual income and the loan amount you are requesting?\n"
                   "user: My income is abc, and I need 80,000.\n"
                   "assistant: Your income must be a numeric value. Could you provide it correctly?\n"
                   "user: Sorry, it’s 50000.\n"
                   "assistant: Great! What is the value of your property?\n"
                   "user: It’s worth xyz.\n"
                   "assistant: The property value must be numeric. Could you provide the correct value?\n"
                   "user: Okay, it’s 300000.\n"
                   "assistant: Thanks! Lastly, what is the loan term in months?\n"
                   "Latest user message: 15 years."
      },
      {"role": "assistant", "content": "{\"name\": \"John\", \"income\": 50000, \"loan_amount\": 80000, \"property_value\": 300000, \"loan_term\": 180, \"loan_purpose\": \"Home improvement\"}"}
    ]
  },
]

In [56]:
len(examples)

41

In [55]:
import json
with open("train_llm_data.jsonl", "w") as f:
    for example in examples:
        f.write(json.dumps(example) + "\n")

In [57]:
!cat train_llm_data.jsonl | grep 'messages'

{"messages": [{"role": "system", "content": "Given the mandatory fields: \"name\", \"income\", \"loan_amount\", \"property_value\", \"loan_term\", and \"loan_purpose\", validate and collect missing or invalid fields. Return a JSON object with the completed fields."}, {"role": "user", "content": "Hi, I'm John, and I earn 80000 annually. I want a loan of 250000 for my new home."}, {"role": "assistant", "content": "{\"name\": \"John\", \"income\": 80000, \"loan_amount\": 250000, \"property_value\": null, \"loan_term\": null, \"loan_purpose\": \"Home purchase\"}"}]}
{"messages": [{"role": "system", "content": "Given the mandatory fields: \"name\", \"income\", \"loan_amount\", \"property_value\", \"loan_term\", and \"loan_purpose\", validate and collect missing or invalid fields. Return a JSON object with the completed fields."}, {"role": "user", "content": "I\u2019m Sarah, and I need a loan for 150000. My property is valued at 500000. The loan term will be 30 years, and it\u2019s for home 

# Upload file

In [ ]:
client = OpenAI()

upload_file_response = client.files.create(
  file=open("train_llm_data.jsonl", "rb"),
  purpose="fine-tune"
)
upload_file_response

FileObject(id='file-MPgDA4PwaX84DammvxbQBe', bytes=24892, created_at=1736260266, filename='train_llm_data.jsonl', object='file', purpose='fine-tune', status='processed', status_details=None)

## Create a file tune model

In [60]:
create_job_response = client.fine_tuning.jobs.create(
  training_file=upload_file_response.id,
  model="gpt-4o-mini-2024-07-18",
  suffix='homeloan_llm',
  hyperparameters={
    "n_epochs":10
  }
)
create_job_response

FineTuningJob(id='ftjob-ETvCNKRQL9sFw9z2eoOYLrsN', created_at=1736260282, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs=10, batch_size='auto', learning_rate_multiplier='auto'), model='gpt-4o-mini-2024-07-18', object='fine_tuning.job', organization_id='org-gpaASmvfepNl5rppSKmtZJKZ', result_files=[], seed=123011109, status='validating_files', trained_tokens=None, training_file='file-MPgDA4PwaX84DammvxbQBe', validation_file=None, estimated_finish=None, integrations=[], user_provided_suffix='homeloan_llm', method={'type': 'supervised', 'supervised': {'hyperparameters': {'batch_size': 'auto', 'learning_rate_multiplier': 'auto', 'n_epochs': 10}}})

In [61]:
import openai

# List 10 fine-tuning jobs
list_jobs = openai.fine_tuning.jobs.list(limit=10)
list_jobs

SyncCursorPage[FineTuningJob](data=[FineTuningJob(id='ftjob-ETvCNKRQL9sFw9z2eoOYLrsN', created_at=1736260282, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs=10, batch_size='auto', learning_rate_multiplier='auto'), model='gpt-4o-mini-2024-07-18', object='fine_tuning.job', organization_id='org-gpaASmvfepNl5rppSKmtZJKZ', result_files=[], seed=123011109, status='validating_files', trained_tokens=None, training_file='file-MPgDA4PwaX84DammvxbQBe', validation_file=None, estimated_finish=None, integrations=[], user_provided_suffix='homeloan_llm', method={'type': 'supervised', 'supervised': {'hyperparameters': {'n_epochs': 10, 'batch_size': 'auto', 'learning_rate_multiplier': 'auto'}}}), FineTuningJob(id='ftjob-DJYWSK8qoa2ELKI80xmHHxt0', created_at=1736257633, error=Error(code=None, message=None, param=None), fine_tuned_model='ft:gpt-4o-mini-2024-07-18:personal:homeloan-llm:An4NwIuu', finished_at=1736258290, hy

In [62]:
openai.fine_tuning.jobs.retrieve(list_jobs.data[0].id)

FineTuningJob(id='ftjob-ETvCNKRQL9sFw9z2eoOYLrsN', created_at=1736260282, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs=10, batch_size='auto', learning_rate_multiplier='auto'), model='gpt-4o-mini-2024-07-18', object='fine_tuning.job', organization_id='org-gpaASmvfepNl5rppSKmtZJKZ', result_files=[], seed=123011109, status='validating_files', trained_tokens=None, training_file='file-MPgDA4PwaX84DammvxbQBe', validation_file=None, estimated_finish=None, integrations=[], user_provided_suffix='homeloan_llm', method={'type': 'supervised', 'supervised': {'hyperparameters': {'n_epochs': 10, 'batch_size': 'auto', 'learning_rate_multiplier': 'auto'}}})

In [63]:
# List up to 10 events from a fine-tuning job
list_events = openai.fine_tuning.jobs.list_events(fine_tuning_job_id=list_jobs.data[0].id, limit=10)
for d in list_events.data:
    print(d)

FineTuningJobEvent(id='ftevent-ae5yhmgUENMJDy10QiQdgcP5', created_at=1736260282, level='info', message='Validating training file: file-MPgDA4PwaX84DammvxbQBe', object='fine_tuning.job.event', data={}, type='message')
FineTuningJobEvent(id='ftevent-q0vHfGLwbXuOQ29HWJ0a3cbU', created_at=1736260282, level='info', message='Created fine-tuning job: ftjob-ETvCNKRQL9sFw9z2eoOYLrsN', object='fine_tuning.job.event', data={}, type='message')


### Use fine-tune model

In [34]:
# history = 
completion = openai.chat.completions.create(
  model="gpt-4o-mini",
  messages=[
    {"role": "system", "content": "Given the home loan mandatory fields: \"name\", \"income\", \"loan_amount\", \"property_value\", \"loan_term\", and \"loan_purpose\", validate and collect missing or invalid fields. Return a JSON object with the completed fields"},
    {"role": "user", "content": "My name is Alex, and I earn 120k a year. I need 200k for a cash-out refinancing on my property worth 650000 for a loan term 360"}
  ]
)
print(completion.choices[0].message)

ChatCompletionMessage(content='Based on the provided information, here’s the breakdown of the mandatory fields:\n\n- **name**: Alex (provided)\n- **income**: 120k (provided)\n- **loan_amount**: 200k (provided)\n- **property_value**: 650000 (provided)\n- **loan_term**: 360 (provided)\n- **loan_purpose**: (missing)\n\nThe only missing field is "loan_purpose." \n\nHere’s the JSON object with the completed fields, including a placeholder for the missing "loan_purpose":\n\n```json\n{\n  "name": "Alex",\n  "income": 120000,\n  "loan_amount": 200000,\n  "property_value": 650000,\n  "loan_term": 360,\n  "loan_purpose": "cash-out refinancing" // Use if loan_purpose is known; otherwise, update accordingly\n}\n```\n\nPlease provide the specific loan purpose or confirm if it is "cash-out refinancing" to finalize the JSON object.', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None)


In [35]:
# history = 
completion = openai.chat.completions.create(
  model="ft:gpt-4o-mini-2024-07-18:personal:homeloan-llm:AmlYZXVx",
  messages=[
    {"role": "system", "content": "Given the home loan mandatory fields: \"name\", \"income\", \"loan_amount\", \"property_value\", \"loan_term\", and \"loan_purpose\", validate and collect missing or invalid fields. Return a JSON object with the completed fields"},
    {"role": "user", "content": "My name is Alex, and I want to apply my home loan profiles for 15 years as loan term, abc as income, 500000 as loan amount"}
  ]
)
print(completion.choices[0].message.content)

{"name": "Alex", "income": null, "loan_amount": 500000, "property_value": null, "loan_term": 180, "loan_purpose": null}


## Evaluate fine_tune model

In [10]:
def generate_assistant_output(model, user_inputs):
    completion = openai.chat.completions.create(
    model=model,
    messages=[
            {"role": "system", "content": "Given the home loan mandatory fields: \"name\", \"income\", \"loan_amount\", \"property_value\", \"loan_term\", and \"loan_purpose\", validate and collect missing or invalid fields. Return a JSON object with the completed fields"},
            {"role": "user", "content": user_inputs}
        ]    
    )
    return completion.choices[0].message.content

In [2]:
test_dataset = [
    {
        "messages": [
            {"role": "user", "content": "Hi, my name is Alice. I make $75,000 annually and want a loan of $300,000 for a home purchase."},
            {"role": "assistant", "content": "{\"name\": \"Alice\", \"income\": 75000, \"loan_amount\": 300000, \"property_value\": null, \"loan_term\": null, \"loan_purpose\": \"Home purchase\"}"}
        ]
    },
    {
        "messages": [
            {"role": "user", "content": "I need a loan of $200,000 for a property worth $400,000. My income is $60k annually."},
            {"role": "assistant", "content": "{\"name\": null, \"income\": 60000, \"loan_amount\": 200000, \"property_value\": 400000, \"loan_term\": null, \"loan_purpose\": null}"}
        ]
    },
    {
        "messages": [
            {"role": "user", "content": "I’m Robert, and I earn around 120k annually. I want a refinance loan for my $800,000 house."},
            {"role": "assistant", "content": "{\"name\": \"Robert\", \"income\": 120000, \"loan_amount\": null, \"property_value\": 800000, \"loan_term\": null, \"loan_purpose\": \"Refinance\"}"}
        ]
    },
    {
        "messages": [
            {"role": "user", "content": "I need $100,000 for home improvement. My property is worth $500,000."},
            {"role": "assistant", "content": "{\"name\": null, \"income\": null, \"loan_amount\": 100000, \"property_value\": 500000, \"loan_term\": null, \"loan_purpose\": \"Home improvement\"}"}
        ]
    },
    {
        "messages": [
            {"role": "user", "content": "My name is Maria. I earn $90,000 and need a $250,000 loan for a new home."},
            {"role": "assistant", "content": "{\"name\": \"Maria\", \"income\": 90000, \"loan_amount\": 250000, \"property_value\": null, \"loan_term\": null, \"loan_purpose\": \"Home purchase\"}"}
        ]
    },
    {
        "messages": [
            {"role": "user", "content": "Hi, I’m Liam. My house is worth $700,000. I want to refinance."},
            {"role": "assistant", "content": "{\"name\": \"Liam\", \"income\": null, \"loan_amount\": null, \"property_value\": 700000, \"loan_term\": null, \"loan_purpose\": \"Refinance\"}"}
        ]
    },
    {
        "messages": [
            {"role": "user", "content": "I earn $50,000 per year and need a $100,000 loan for repairs."},
            {"role": "assistant", "content": "{\"name\": null, \"income\": 50000, \"loan_amount\": 100000, \"property_value\": null, \"loan_term\": null, \"loan_purpose\": \"Home improvement\"}"}
        ]
    },
    {
        "messages": [
            {"role": "system", "content": "Given the mandatory fields, ensure all fields are validated and collected."},
            {"role": "user", "content": "I want a $500,000 loan for cash-out refinancing. My property is worth $1 million."},
            {"role": "assistant", "content": "{\"name\": null, \"income\": null, \"loan_amount\": 500000, \"property_value\": 1000000, \"loan_term\": null, \"loan_purpose\": \"Cash-out refinancing\"}"}
        ]
    },
    {
        "messages": [
            {"role": "user", "content": "My name is Emma. I don’t know my exact income but it’s around $80,000. I need a $200,000 loan for home improvement."},
            {"role": "assistant", "content": "{\"name\": \"Emma\", \"income\": 80000, \"loan_amount\": 200000, \"property_value\": null, \"loan_term\": null, \"loan_purpose\": \"Home improvement\"}"}
        ]
    },
    {
        "messages": [
            {"role": "user", "content": "I think my income is $100k. I want to refinance my $700,000 house for $300,000."},
            {"role": "assistant", "content": "{\"name\": null, \"income\": 100000, \"loan_amount\": 300000, \"property_value\": 700000, \"loan_term\": null, \"loan_purpose\": \"Refinance\"}"}
        ]
    },
    {
        "messages": [
            {"role": "user", "content": "I need $250,000 for a property worth $800,000."},
            {"role": "assistant", "content": "{\"name\": null, \"income\": null, \"loan_amount\": 250000, \"property_value\": 800000, \"loan_term\": null, \"loan_purpose\": null}"}
        ]
    },
    {
        "messages": [
            {"role": "user", "content": "My house is worth $500,000. I’d like $100,000 for home improvement."},
            {"role": "assistant", "content": "{\"name\": null, \"income\": null, \"loan_amount\": 100000, \"property_value\": 500000, \"loan_term\": null, \"loan_purpose\": \"Home improvement\"}"}
        ]
    },
    {
        "messages": [
            {"role": "user", "content": "I’m David. I earn $120,000 and want a $400,000 loan for a property worth $1.2 million."},
            {"role": "assistant", "content": "{\"name\": \"David\", \"income\": 120000, \"loan_amount\": 400000, \"property_value\": 1200000, \"loan_term\": null, \"loan_purpose\": null}"}
        ]
    },
    {
        "messages": [
            {"role": "user", "content": "I’m Tom. I want a $200,000 loan for home improvement. My house is valued at $600,000."},
            {"role": "assistant", "content": "{\"name\": \"Tom\", \"income\": null, \"loan_amount\": 200000, \"property_value\": 600000, \"loan_term\": null, \"loan_purpose\": \"Home improvement\"}"}
        ]
    },
    {
        "messages": [
            {"role": "user", "content": "I make $50k per year. My property is worth $300,000. I need $150,000 for home improvement."},
            {"role": "assistant", "content": "{\"name\": null, \"income\": 50000, \"loan_amount\": 150000, \"property_value\": 300000, \"loan_term\": null, \"loan_purpose\": \"Home improvement\"}"}
        ]
    },
    {
        "messages": [
            {"role": "user", "content": "I’d like a loan for $400,000 for a $1 million property. My income is $150,000."},
            {"role": "assistant", "content": "{\"name\": null, \"income\": 150000, \"loan_amount\": 400000, \"property_value\": 1000000, \"loan_term\": null, \"loan_purpose\": null}"}
        ]
    },
    {
        "messages": [
            {"role": "user", "content": "I’m Sarah. My income is $95k. I need $300,000 to buy a house worth $1.5 million."},
            {"role": "assistant", "content": "{\"name\": \"Sarah\", \"income\": 95000, \"loan_amount\": 300000, \"property_value\": 1500000, \"loan_term\": null, \"loan_purpose\": \"Home purchase\"}"}
        ]
    },
    {
        "messages": [
            {"role": "user", "content": "I need $200,000 for a house worth $600,000."},
            {"role": "assistant", "content": "{\"name\": null, \"income\": null, \"loan_amount\": 200000, \"property_value\": 600000, \"loan_term\": null, \"loan_purpose\": null}"}
        ]
    },
    {
        "messages": [
            {"role": "user", "content": "My income is $80,000. I want a $250,000 loan for my house worth $900,000."},
            {"role": "assistant", "content": "{\"name\": null, \"income\": 80000, \"loan_amount\": 250000, \"property_value\": 900000, \"loan_term\": null, \"loan_purpose\": null}"}
        ]
    }
] + [
    {
        "messages": [
            {
                "role": "user", 
                "content": 
                           "Chat History : "
                           "user: Hi, I’m Mike, and I need to refinance my house.\n"
                           "assistant: Hi Mike! Could you provide your yearly income and the loan amount?\n"
                           "user: My yearly earnings are abc, and I want 250,000.\n"
                           "assistant: Your income must be a numeric value. Can you confirm it?\n"
                           "user: Oh, my income is 85,000.\n"
                           "assistant: Great! Could you also provide the value of your property?\n"
                           "user: I think it’s worth somewhere around xyz.\n"
                           "assistant: The property value must be numeric. Please provide an accurate amount.\n"
                           "user: Got it, it’s 400,000.\n"
                           "assistant: Lastly, what’s the loan term in years or months?\n"
                           "Latest user message: 15 years."
            },
            {"role": "assistant", "content": "{\"name\": \"Mike\", \"income\": 85000, \"loan_amount\": 250000, \"property_value\": 400000, \"loan_term\": 180, \"loan_purpose\": \"Refinance\"}"}
        ]
    },
    {
        "messages": [
      {
          "role": "user", 
          "content": 
                           "Chat History : "
                           "user: Hello, I’m Sophia, and I want to apply for a loan to buy a house.\n"
                           "assistant: Hi Sophia! Can you share your annual income and the loan amount you’re seeking?\n"
                           "user: My salary is abc annually, and I’d like 300,000.\n"
                           "assistant: The income must be numeric. Can you provide it correctly?\n"
                           "user: Sure, it’s 92,000.\n"
                           "assistant: Could you also share the estimated value of the house?\n"
                           "user: It’s valued at around xyz.\n"
                           "assistant: Property value must be numeric. Could you provide the exact number?\n"
                           "user: It’s 1,000,000.\n"
                           "assistant: Finally, how long is the loan term you are considering?\n"
                           "Latest user message: 25 years."
            },
            {"role": "assistant", "content": "{\"name\": \"Sophia\", \"income\": 92000, \"loan_amount\": 300000, \"property_value\": 1000000, \"loan_term\": 300, \"loan_purpose\": \"Home purchase\"}"}
        ]
    },
    {
        "messages": [
      {
          "role": "user", 
          "content": 
                           "Chat History : "
                           "user: Hi, my name is Emma. I’m looking for a loan to improve my home.\n"
                           "assistant: Hi Emma! Can you tell me your annual income and the loan amount?\n"
                           "user: My income is approximately abc, and I need 150,000.\n"
                           "assistant: Income must be numeric. Please confirm the correct value.\n"
                           "user: It’s 75,000.\n"
                           "assistant: Thanks! What’s the value of your property?\n"
                           "user: I’m guessing it’s around xyz.\n"
                           "assistant: Property value must be numeric. Could you confirm the exact amount?\n"
                           "user: Okay, it’s 650,000.\n"
                           "assistant: Lastly, what loan term are you thinking?\n"
                           "Latest user message: 10 years."
            },
            {"role": "assistant", "content": "{\"name\": \"Emma\", \"income\": 75000, \"loan_amount\": 150000, \"property_value\": 650000, \"loan_term\": 120, \"loan_purpose\": \"Home improvement\"}"}
        ]
    },
    {
        "messages": [
            {
                "role": "user", 
                "content": 
                           "Chat History : "
                           "user: I’m Alex, and I’d like to refinance my house.\n"
                           "assistant: Hi Alex! Can you tell me your annual income and how much you’d like to borrow?\n"
                           "user: My income is abc, and I need 200,000.\n"
                           "assistant: Income must be numeric. Can you confirm the actual value?\n"
                           "user: Sorry, it’s 110,000.\n"
                           "assistant: Thank you. What is the value of your property?\n"
                           "user: It’s worth xyz.\n"
                           "assistant: The property value must be numeric. Please provide a specific value.\n"
                           "user: It’s 750,000.\n"
                           "assistant: Finally, what’s the loan term you’re considering?\n"
                           "Latest user message: 20 years."
            },
            {"role": "assistant", "content": "{\"name\": \"Alex\", \"income\": 110000, \"loan_amount\": 200000, \"property_value\": 750000, \"loan_term\": 240, \"loan_purpose\": \"Refinance\"}"}
        ]
    },
    {
        "messages": [
            {
                "role": "user", 
                "content": 
                           "Chat History : "
                           "user: Hi, I’m Liam. I need a loan to pay off my current mortgage and renovate my home.\n"
                           "assistant: Hi Liam! Can you share your annual income and the amount you’d like to borrow?\n"
                           "user: I make about abc annually, and I need 180,000.\n"
                           "assistant: Income must be numeric. Could you confirm the actual amount?\n"
                           "user: It’s 65,000.\n"
                           "assistant: What’s the estimated value of your property?\n"
                           "user: I believe it’s xyz.\n"
                           "assistant: The property value must be numeric. Could you provide a specific number?\n"
                           "user: Sure, it’s 500,000.\n"
                           "assistant: Finally, what’s the term of the loan in years or months?\n"
                           "Latest user message: 15 years."
            },
            {"role": "assistant", "content": "{\"name\": \"Liam\", \"income\": 65000, \"loan_amount\": 180000, \"property_value\": 500000, \"loan_term\": 180, \"loan_purpose\": \"Refinance\"}"}
        ]
    }
]

In [3]:
print(len(test_dataset))

24


In [11]:
import json

model_finetune ="ft:gpt-4o-mini-2024-07-18:personal:homeloan-llm:An55Uasr"
pd_data_set = {}

correct = 0
true_labels = []
predicted_labels = []

for ele in test_dataset:
    user_input = ele['messages'][0]['content']
    label = ele['messages'][1]['content']
    assistant_output = generate_assistant_output(model_finetune, user_input)
    print(f"question : {user_input}")
    print(f"label    : {label}")
    print(f"assistant output : {assistant_output}")
    if label == assistant_output:
        print("pass")
    else:
        print("failed")
    correct += 1 if label == assistant_output else 0

question : Hi, my name is Alice. I make $75,000 annually and want a loan of $300,000 for a home purchase.
label    : {"name": "Alice", "income": 75000, "loan_amount": 300000, "property_value": null, "loan_term": null, "loan_purpose": "Home purchase"}
assistant output : {"name": "Alice", "income": 75000, "loan_amount": 300000, "property_value": null, "loan_term": null, "loan_purpose": "Home purchase"}
pass
question : I need a loan of $200,000 for a property worth $400,000. My income is $60k annually.
label    : {"name": null, "income": 60000, "loan_amount": 200000, "property_value": 400000, "loan_term": null, "loan_purpose": null}
assistant output : {"name": null, "income": 60000, "loan_amount": 200000, "property_value": 400000, "loan_term": null, "loan_purpose": null}
pass
question : I’m Robert, and I earn around 120k annually. I want a refinance loan for my $800,000 house.
label    : {"name": "Robert", "income": 120000, "loan_amount": null, "property_value": 800000, "loan_term": null,

In [12]:
accuracy = (correct / len(test_dataset)) * 100
print(f"Accuracy: {accuracy:.2f}%")

Accuracy: 95.83%


In [18]:
question = "I’m Peter, and I earn 80000 annually. I need 500000. My property is valued at 450000. and I need a loan for 360 months."
assistant_output = generate_assistant_output(model_finetune, question)
print(assistant_output)

{"name": "Peter", "income": 80000, "loan_amount": 500000, "property_value": 450000, "loan_term": 360, "loan_purpose": null}
